In [1]:
from src.explainer.explainer import ArgumentativeExplainer
from src.explainer.framework import ArgumentationFramework

from src.explainer.adjective import BooleanAdjective, PointerAdjective, QuantitativePointerAdjective, NodesGroupPointerAdjective, ComparisonAdjective, MaxRankAdjective, MinRankAdjective
from src.explainer.explanation import Possession, Assumption, If, ConditionalExplanation, CompositeExplanation

from src.explainer.explanation_tactics import OnlyRelevantComparisons, SkipQuantitativeExplanations, SubstituteQuantitativeExplanations, SkipConditionStatement

## We will utilize this example search tree:

In [2]:
class MinMaxNode:
    def __init__(self, id, *, score=None, maximizing_player_turn=True, children=None, score_child=None):
        
        self.id = id

        if score:
            self.score = score
        elif score_child:
            self.score_child = score_child
            self.score = score_child.score
        else:
            raise ValueError("Provide score or score_child.")

        self.children = children or []
        self.parent = None
        self.maximizing_player_turn = maximizing_player_turn
        
        self.is_leaf = True
        if len(self.children) > 0:
            self.is_leaf = False
            for child in children:
                child.parent=self
                child.maximizing_player_turn = not self.maximizing_player_turn
    
    def __str__(self):
        return self.id

# Create a simple game tree
leaf11 = MinMaxNode('leaf11', score=3)
leaf12 = MinMaxNode('leaf12', score=4)
leaf21 = MinMaxNode('leaf21', score=8)
leaf22 = MinMaxNode('leaf22', score=2)
leaf31 = MinMaxNode('leaf21', score=1)
leaf32 = MinMaxNode('leaf22', score=1)

child1 = MinMaxNode('child1', children=[leaf11, leaf12], score_child=leaf11)
#child2 = MinMaxNode('child2', children=[leaf21, leaf22], score_child=leaf22)
child2 = MinMaxNode('child2', score = 2)
child3 = MinMaxNode('child3', children=[leaf31, leaf32], score_child=leaf31)

root = MinMaxNode('root', maximizing_player_turn=True, children=[child1, child2, child3], score_child=child1)

## Framework definition

### We will make two frameworks: one for Low abstraction Level explanations, and another for High abstraction Level explanations.

We will design them so that:

- The Low abstraction framework can explain more closely how the search tree algorithm works and reasons.
- The High abstraction framework can explain moves on a more abstract game concept level.

### Low Level explanation framework:

In [3]:
knowledgebase_ll = ArgumentationFramework(refer_to_nodes_as = 'node')

knowledgebase_ll.add_adjectives([
    
    BooleanAdjective("leaf",
        definition = "node.is_leaf"),


    QuantitativePointerAdjective("score",
        definition = "node.score",

        explanation = ConditionalExplanation(
            condition = If("leaf"),
            explanation_if_true = Assumption("Leaf nodes have scores from the evaluation function"),
            explanation_if_false = CompositeExplanation(
                Assumption("Internal nodes have scores from children"),
                Possession("backpropagating child"))
        )),


    BooleanAdjective("opponent player turn",
        definition = "not node.maximizing_player_turn"),


    PointerAdjective("backpropagating child",
        definition = "node.score_child",

        explanation = ConditionalExplanation(
            condition = If("opponent player turn"),
            explanation_if_true = CompositeExplanation(
                Assumption("We assume the opponent will do their best move."),
                Possession("backpropagating child", "worst")),
            explanation_if_false = CompositeExplanation(
                Assumption("On our turn we take the maximum rated move."),
                Possession("backpropagating child", "best"))
        )),

    ComparisonAdjective("better", "score", ">"),
 
    NodesGroupPointerAdjective("siblings",
        definition = "node.parent.children",
        excluding = "node"),

    MaxRankAdjective("best", "better", "siblings"),

    MinRankAdjective("worst", "better", "siblings"),
])

### High Level explanation framework

In [4]:
knowledgebase_hl = ArgumentationFramework(refer_to_nodes_as = 'move')

knowledgebase_hl.add_adjectives([
    
    BooleanAdjective("final move",
        definition = "node.is_leaf"),


    QuantitativePointerAdjective("score",
        definition = "node.score",

        explanation = ConditionalExplanation(
            condition = If("final move"),
            explanation_if_true = Assumption("final moves are evaluated only looking at the final position", necessary=True),
            explanation_if_false = CompositeExplanation(
                Possession("next possible move"))
        )),
    

    BooleanAdjective("opponent player turn",
        definition = "not node.maximizing_player_turn"),


    PointerAdjective("next possible move",
        definition = "node.score_child",

        explanation = ConditionalExplanation(
            condition = If("opponent player turn"),
            explanation_if_true = CompositeExplanation(
                Assumption("we assume the opponent will do their best move"),
                Possession("next possible move", "the best the opponent can do")),
            explanation_if_false = CompositeExplanation(
                Assumption("on our turn we take the maximum rated move"),
                Possession("next possible move", "the best"))
        )),

    ComparisonAdjective("better", "score", ">"),
 
    NodesGroupPointerAdjective("possible alternative moves",
        definition = "node.parent.children",
        excluding = "node"),

    MaxRankAdjective("the best", "better", "possible alternative moves"),

    MinRankAdjective("the best the opponent can do", "better", "possible alternative moves"),
])

### Notice the differences:

In [5]:
print(knowledgebase_ll)

Propositions:
node is leaf
node has score = ?
node is opponent player turn
node has backpropagating child = ?
node1 is better than node2
node has siblings = ?
node is best
node is worst


Implications:
node has leaf = ? ←
 (assumption) Definition of "leaf" is node.is_leaf
node has score = ? ←
 node is leaf ←
 (assumption) Definition of "leaf" is node.is_leaf
∧ (assumption) Leaf nodes have scores from the evaluation function
|| node is ¬(leaf) ←
 (assumption) Definition of "leaf" is node.is_leaf
∧ (assumption) Internal nodes have scores from children
∧ node has backpropagating child = ? ←
 node is opponent player turn ←
 (assumption) Definition of "opponent player turn" is not node.maximizing_player_turn
∧ (assumption) We assume the opponent will do their best move.
∧ node has worst = ? ←
 (assumption) By definition a node is "worst" if it's not "better" than all "siblings"
∧ Node not better than all nodes in siblings
|| node is ¬(opponent player turn) ←
 (assumption) Definition of "opp

In [6]:
print(knowledgebase_hl)

Propositions:
move is final move
move has score = ?
move is opponent player turn
move has next possible move = ?
move1 is better than move2
move has possible alternative moves = ?
move is the best
move is the best the opponent can do


Implications:
move has final move = ? ←
 (assumption) Definition of "final move" is node.is_leaf
move has score = ? ←
 move is final move ←
 (assumption) Definition of "final move" is node.is_leaf
∧ (assumption) final moves are evaluated only looking at the final position
|| move is ¬(final move) ←
 (assumption) Definition of "final move" is node.is_leaf
∧ move has next possible move = ? ←
 move is opponent player turn ←
 (assumption) Definition of "opponent player turn" is not node.maximizing_player_turn
∧ (assumption) we assume the opponent will do their best move
∧ move has the best the opponent can do = ? ←
 (assumption) By definition a move is "the best the opponent can do" if it's not "better" than all "possible alternative moves"
∧ Node not better

## We can now add the frameworks to an ArgumentativeExplainer:

In [7]:
explainer = ArgumentativeExplainer()
explainer.add_framework("lowlevel", knowledgebase_ll)
explainer.add_framework("highlevel", knowledgebase_hl)

#### Add explanation tactics:

Notice that they are applied to specific frameworks and, if necessary, adjectives.

In [8]:
explainer.add_explanation_tactic(OnlyRelevantComparisons(mode = "top_1"), to_adjective="the best", to_framework='highlevel')
explainer.add_explanation_tactic(SubstituteQuantitativeExplanations("it leads to a better position"), to_framework='highlevel')
explainer.add_explanation_tactic(SkipConditionStatement(except_on_adjectives=['score']), to_framework='highlevel')

#### And set the explanations settings:

In [9]:
high_abstraction_settings = {
            'with_framework': 'highlevel',
            'explanation_depth': 4 ,
            'print_implicit_assumptions': False,
            'assumptions_verbosity': 'if_asked',
            'print_mode': 'verbal'
        }

explainer.configure_settings(high_abstraction_settings)

## We can now explain nodes' properties

In [10]:
explainer.explain_adjective(root, "the best", explanation_depth=4)

The adjective "the best" cannot be evaluated on the move root.
That is because The adjective "possible alternative moves" cannot be evaluated on root.


In [11]:
explainer.explain_adjective(child1, "the best")

child1 is the best (because
 	child1 has as possible alternative moves child2 (only showing relevant 1)
	and child1 is better than child2 (because
	 	it leads to a better position (because
	 		child1 is not final move
			and child1 has as next possible move leaf11 (because
			 				leaf11 is the best the opponent can do)
			and child2 is final move
			and (assumption) final moves are evaluated only looking at the final position)))


In [12]:
explainer.explain_adjective(child1, "the best", explanation_depth=5)

child1 is the best (because
 	child1 has as possible alternative moves child2 (only showing relevant 1)
	and child1 is better than child2 (because
	 	it leads to a better position (because
	 		child1 is not final move
			and child1 has as next possible move leaf11 (because
			 				leaf11 is the best the opponent can do (because
							 					leaf11 has as possible alternative moves leaf12 (only showing relevant 1) and leaf11 is not better than leaf12))
			and child2 is final move
			and (assumption) final moves are evaluated only looking at the final position)))


In [13]:
explainer.explain_adjective(child2, "score")

child2 is final move
and (assumption) final moves are evaluated only looking at the final position


In [14]:
explainer.explain_adjective(child1, "the best", explanation_depth=2)

child1 is the best (because
 	child1 has as possible alternative moves child2 (only showing relevant 1) and child1 is better than child2)


In [15]:
explainer.explain_adjective(child1, "better", child2)

child1 is better than child2 (because
 	it leads to a better position (because
 	child1 is not final move
	and child1 has as next possible move leaf11 (because
	 			leaf11 is the best the opponent can do (because
				 				leaf11 has as possible alternative moves leaf12 and leaf11 is not better than leaf12))
	and child2 is final move
	and (assumption) final moves are evaluated only looking at the final position))


### Here you can see the lowlevel framework in action

In [16]:
low_abstraction_settings = {
            'with_framework': 'lowlevel',
            'explanation_depth': 3 ,
            'print_implicit_assumptions': True,
            'assumptions_verbosity': 'verbose',
            'print_mode': 'logic'
        }

explainer.configure_settings(low_abstraction_settings)

In [17]:
explainer.explain_adjective(child1, "worst", explanation_depth = 0)

child1 is ¬(worst)


In [18]:
explainer.explain_adjective(child1, "worst", explanation_depth = 1)

child1 is ¬(worst) ←
 	(assumption) By definition a node is "worst" if it's not "better" than all "siblings"
	∧ child1 has siblings = child2, child3
	∧ child1 is better than child2 ∧ child1 is better than child3


In [19]:
explainer.explain_adjective(child1, "worst")

child1 is ¬(worst) ←
 	(assumption) By definition a node is "worst" if it's not "better" than all "siblings"
	∧ child1 has siblings = child2, child3 ←
	 		(assumption) Definition of "siblings" is node.parent.children excluding node
	∧ child1 is better than child2 ←
	 		(assumption) By definition, node1 is "better" than node2 if node1 score > node2 score
			∧ child1 has score = 3 ←
			 			child1 is ¬(leaf)
						∧ (assumption) Internal nodes have scores from children
						∧ child1 has backpropagating child = leaf11
			∧ child2 has score = 2 ←
			 			child2 is leaf
						∧ (assumption) Leaf nodes have scores from the evaluation function
	∧ child1 is better than child3 ←
	 		(assumption) By definition, node1 is "better" than node2 if node1 score > node2 score
			∧ child1 has score = 3 ←
			 			child1 is ¬(leaf)
						∧ (assumption) Internal nodes have scores from children
						∧ child1 has backpropagating child = leaf11
			∧ child3 has score = 1 ←
			 			child3 is ¬(leaf)
						∧ (assum

### You can also print the depth of arguments

In [20]:
explainer.explain_adjective(child1, "worst", print_depth=True)

child1 is ¬(worst) ←
 	Depth 1:
	(assumption) By definition a node is "worst" if it's not "better" than all "siblings"
	∧ child1 has siblings = child2, child3 ←
	 		Depth 2:
			(assumption) Definition of "siblings" is node.parent.children excluding node
	∧ child1 is better than child2 ←
	 		Depth 2:
			(assumption) By definition, node1 is "better" than node2 if node1 score > node2 score
			∧ child1 has score = 3 ←
			 			Depth 3:
						child1 is ¬(leaf)
						∧ (assumption) Internal nodes have scores from children
						∧ child1 has backpropagating child = leaf11
			∧ child2 has score = 2 ←
			 			Depth 3:
						child2 is leaf
						∧ (assumption) Leaf nodes have scores from the evaluation function
	∧ child1 is better than child3 ←
	 		Depth 2:
			(assumption) By definition, node1 is "better" than node2 if node1 score > node2 score
			∧ child1 has score = 3 ←
			 			Depth 3:
						child1 is ¬(leaf)
						∧ (assumption) Internal nodes have scores from children
						∧ child1 has backpro

In [21]:
explainer.explain_adjective(child1, "better", child2)

child1 is better than child2 ←
 	(assumption) By definition, node1 is "better" than node2 if node1 score > node2 score
	∧ child1 has score = 3 ←
	 		child1 is ¬(leaf) ←
			 			(assumption) Definition of "leaf" is node.is_leaf
			∧ (assumption) Internal nodes have scores from children
			∧ child1 has backpropagating child = leaf11 ←
			 			child1 is opponent player turn
						∧ (assumption) We assume the opponent will do their best move.
						∧ leaf11 is worst
	∧ child2 has score = 2 ←
	 		child2 is leaf ←
			 			(assumption) Definition of "leaf" is node.is_leaf
			∧ (assumption) Leaf nodes have scores from the evaluation function


In [22]:
explainer.explain_adjective(child1, "siblings")

child1 has siblings = child2, child3 ←
 	(assumption) Definition of "siblings" is node.parent.children excluding node


In [23]:
explainer.explain_adjective(root, "backpropagating child")

root has backpropagating child = child1 ←
 	root is ¬(opponent player turn) ←
	 		(assumption) Definition of "opponent player turn" is not node.maximizing_player_turn
	∧ (assumption) On our turn we take the maximum rated move.
	∧ child1 is best ←
	 		(assumption) By definition a node is "best" if it's "better" than all "siblings"
			∧ child1 has siblings = child2, child3 ←
			 			(assumption) Definition of "siblings" is node.parent.children excluding node
			∧ child1 is better than child2 ←
			 			(assumption) By definition, node1 is "better" than node2 if node1 score > node2 score
						∧ child1 has score = 3 ∧ child2 has score = 2
			∧ child1 is better than child3 ←
			 			(assumption) By definition, node1 is "better" than node2 if node1 score > node2 score
						∧ child1 has score = 3 ∧ child3 has score = 1


In [24]:
explainer.explain_adjective(root, "score", explanation_depth = 1, print_depth=True)

root has score = 3 ←
 	Depth 1:
	root is ¬(leaf)
	∧ (assumption) Internal nodes have scores from children
	∧ root has backpropagating child = child1
